<a href="https://colab.research.google.com/github/syedsauda/CE888/blob/master/lab5/my_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import itertools

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import accuracy_score as acc
from sklearn.metrics import make_scorer, confusion_matrix
from sklearn.dummy import DummyRegressor, DummyClassifier
from sklearn.preprocessing import LabelEncoder

#Load data

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/albanda/CE888/master/lab5-recommender/jester-data-1.csv", header=None)
print(df)

arr = df.values
print(arr.shape)

rated = np.where(arr!=99)
print(len(rated[0]), rated[1].shape)

       0      1      2      3      4    ...    96     97     98     99     100
0       74  -7.82   8.79  -9.66  -8.16  ...  99.00  -5.63  99.00  99.00  99.00
1      100   4.08  -0.29   6.36   4.37  ...  -2.14   3.06   0.34  -4.32   1.07
2       49  99.00  99.00  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
3       48  99.00   8.35  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
4       91   8.50   4.61  -4.17  -5.39  ...   1.55   3.11   6.55   1.80   1.60
...    ...    ...    ...    ...    ...  ...    ...    ...    ...    ...    ...
24978  100   0.44   7.43   9.08   2.33  ...   9.03   6.55   8.69   8.79   7.43
24979   91   9.13  -8.16   8.59   9.08  ...  -8.20  -7.23  -8.59   9.13   8.45
24980   39  99.00  99.00  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
24981   37  99.00  99.00  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
24982   72   2.43   2.67  -3.98   4.27  ...  99.00  99.00  99.00  99.00  99.00

[24983 rows x 101 columns]
(24983, 101)
1835357 (18

#Helper Funtion

In [0]:
def replace(orig, percentage=0.1):
  """
  Replaces 'percentage'% of the original values in 'orig' with 99's
  :param orig: original data array
  :param percentage: percentage of values to replace (0<percentage<1)
  """
  new_data = orig.copy()
  rated = np.where(arr!=99)
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])

#Replace 10% of random data cells with 99

In [4]:
new_arr, idx = replace(arr, 0.1)
#idx is index of changed data cells

print(arr[idx[0][0], idx[1][0]])
print(new_arr[idx[0][0], idx[1][0]])

36.0
99.0


#LFM to infer hidden ratings

In [0]:
n_latent_factors = 2

user_ratings = arr
# Initialise as random values
latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))


In [0]:
def predict_rating(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)


def train(user_id, item_id, rating, alpha=0.0001):
    
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  prediction_rating - rating
    #print err
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    

def sgd(iterations=20):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = user_ratings[user_id][item_id]
                if rating !=99:
                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean() 
        print(iteration)  
        if (iteration % 10) == 0:
            print(mse)

In [36]:
sgd()  # Note how the MSE decreases with the number of iterations

0
57.92551507239051
1
2
3
4
5
6
7
8
9
10
24.644821101472818
11
12
13
14
15
16
17
18
19


In [37]:
predictions = latent_user_preferences.dot(latent_item_features.T)
predictions

array([[ 7.50170708e+01,  6.84086907e-01,  1.16355012e-01, ...,
         5.16133668e-01, -2.81216086e-01,  8.40754464e-01],
       [ 9.64157169e+01,  1.04421461e+00,  3.08143843e-01, ...,
         9.02231708e-01, -4.13525011e-02,  1.37190600e+00],
       [ 5.46234215e+01,  5.24028986e-01,  1.09632861e-01, ...,
         4.13337802e-01, -1.54494918e-01,  6.57947904e-01],
       ...,
       [ 3.78776231e+01,  6.50708879e-01,  3.52219817e-01, ...,
         7.02611324e-01,  4.50283702e-01,  9.63581870e-01],
       [ 3.75170235e+01,  4.42793995e-01,  1.54962896e-01, ...,
         4.03877232e-01,  5.46640318e-02,  5.98231064e-01],
       [ 7.44727947e+01,  9.46818895e-01,  3.72833362e-01, ...,
         8.99951332e-01,  2.40147795e-01,  1.30732443e+00]])

In [0]:
values = [zip(user_ratings[i], predictions[i]) for i in range(predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data.columns = df.columns

In [39]:
comparison_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,"(74.0, 75.01707078045705)","(-7.82, 0.6840869065762639)","(8.79, 0.1163550119192024)","(-9.66, 0.1490200779753331)","(-8.16, -1.4292850681181184)","(-7.52, 0.3244036101006532)","(-8.5, 1.1547987164247517)","(-9.85, -0.5250784078237615)","(4.17, -0.5870592631197861)","(-8.98, -0.6953300976599761)","(-4.76, 0.9813491382532054)","(-8.5, 1.4504027360040292)","(-6.75, 1.1699210781464429)","(-7.18, -1.8067083974795346)","(8.45, 1.0960506819855134)","(-7.18, -1.6936875145298553)","(-7.52, -3.0643921448599136)","(-7.43, -1.1214836838989637)","(-9.81, -0.7536077316340043)","(-9.85, -0.028300715804347957)","(-9.85, -1.0965769373314382)","(-9.37, 1.885643887737631)","(1.5, 0.6333786289105806)","(-4.37, -0.059113603857680774)","(-9.81, -1.6078577098517963)","(-8.5, 0.10651223412373459)","(1.12, 0.9728227896643247)","(7.82, 2.9094806235780224)","(2.86, 1.1680231192123554)","(9.13, 2.765623936882072)","(-7.43, -0.6766569097087339)","(2.14, 1.9260806084498339)","(-4.08, 2.8587165024347665)","(-9.08, -1.3215430460161361)","(7.82, 0.6589069358480505)","(5.05, 2.7356779237227404)","(4.95, 3.0121732772100924)","(-9.17, -1.3332798452784484)","(-8.4, 0.9824551945289064)","(-8.4, 0.7798884546961616)",...,"(8.59, 2.133213039415614)","(3.59, 2.6258570362026656)","(-6.84, -0.027905065544339743)","(-9.03, -0.7447033618768587)","(2.82, 1.9991746233612504)","(-1.36, 2.219281498479122)","(-9.08, -0.9383049683119624)","(8.3, 2.2915576067574044)","(5.68, 2.3172481823683353)","(-4.81, 0.1970009686682927)","(99.0, -0.7755160181802904)","(99.0, 2.13187254269528)","(99.0, 0.6701806283300282)","(99.0, -1.2481767329437223)","(99.0, -0.23301229238383936)","(99.0, 1.901827959143396)","(99.0, 0.4728247689923197)","(-9.42, 1.2536389585595746)","(99.0, -0.017967265873915242)","(99.0, 0.7835188254983296)","(99.0, 1.3549704247727548)","(-7.72, 0.6548126650782837)","(99.0, 1.594499937055973)","(99.0, 0.47744110043163485)","(99.0, 0.5953147437481076)","(99.0, 0.2417847663181389)","(99.0, 1.388238213067159)","(99.0, 1.5914725603425368)","(99.0, 2.7270055928119743)","(99.0, 0.3657274771640784)","(2.82, 1.4677217286437563)","(99.0, 0.8980569438580087)","(99.0, 1.9100823184217426)","(99.0, 0.8657943178433964)","(99.0, 0.729825411097362)","(99.0, 1.1055520342348628)","(-5.63, 1.2540474864797306)","(99.0, 0.5161336683917227)","(99.0, -0.2812160858348665)","(99.0, 0.8407544642116066)"
1,"(100.0, 96.41571687597018)","(4.08, 1.044214610183464)","(-0.29, 0.3081438428646714)","(6.36, 0.3698860834739804)","(4.37, -1.583313127420061)","(-2.38, 0.5276208273538795)","(-9.66, 1.7484208934977645)","(-0.73, -0.4781667769808764)","(-5.34, -0.6734262757591425)","(8.88, -0.6848570236293476)","(9.22, 1.4633060993198175)","(6.75, 2.0961723013580302)","(8.64, 1.733884259051316)","(4.42, -2.18943467934743)","(7.43, 1.6555154922427189)","(4.56, -2.1159209824176246)","(-0.97, -3.790884166126575)","(4.66, -1.3293032748871219)","(-0.68, -0.8142130220707029)","(3.3, 0.17420203540866935)","(-1.21, -1.2280813836115585)","(0.87, 2.6450583559656113)","(8.64, 1.0449802559432646)","(8.35, 0.1279007646759893)","(9.17, -1.8856038966329725)","(0.05, 0.43368976388808156)","(7.57, 1.518309890541)","(4.71, 3.9074823766921516)","(0.87, 1.778742376433519)","(-0.39, 3.6989310591653384)","(6.99, -0.5727438522070568)","(6.5, 2.7126658833428317)","(-0.92, 3.8707031752984586)","(7.14, -1.5146136101811947)","(9.03, 1.0785645453567567)","(-1.8, 3.710178441267671)","(0.73, 4.058568773856548)","(7.09, -1.5370419108231363)","(3.4, 1.5152383224931105)","(-0.87, 1.295472792453934)",...,"(-6.7, 2.9990075548640873)","(-3.35, 3.630684383333955)","(-9.03, 0.21900214359036888)","(4.47, -0.7305859029036537)","(4.08, 2.8398020162998314)","(-3.83, 3.096425452935731)","(8.74, -0.9703306082948958)","(1